In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import csv
import sys, os
sys.path.append('~/dsi/capstones/cap_3/')

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
import pyspark as ps

import scipy as stats
import re
import dateparser
import datetime
import math

import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols

pd.set_option('display.max_columns', None)


In [110]:
df_org = pd.read_csv('output.csv')

In [111]:
df = df_org.copy()

In [112]:
df.head(5)

,Unnamed: 0,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple,is_tablet,is_watch,sensor_accelerometer,sensor_gyro,sensor_heart_rate,sensor_fingerprint,sensor_compass,sensor_proximity,sensor_barometer,sensor_spo2,sensor_iris_scanner,sensor_gesture,sensor_tempurature,sensor_altimeter,sensor_infrared_face_recognition
0,0,Acer,Chromebook Tab 10,NaN,NaN,NaN,NaN,2018-03,544.3 g (1.20 lb),NaN,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,NaN,"3.1, Type-C 1.0 reversible connector",Accelerometer,About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","13 MP, AF",Yes,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False
2,2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","13 MP, AF",Yes,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Fingerprint (front-mounted), accelerometer, pr...",About 250 EUR,HSDPA,LTE,NaN,Mali-T720MP3,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False
3,3,Acer,Liquid Z6,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,126 g (4.44 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"IPS LCD capacitive touchscreen, 16M colors","5.0 inches, 68.9 cm (~65.3% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6737 (28 nm),Quad-core 1.25 GHz Cortex-A53,"microSD, up to 256 GB","8 MP, AF",Yes,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 120 EUR,HSDPA,LTE,NaN,Mali-T720MP2,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,Fa

In [113]:
drop_cols = ['Unnamed: 0']

df.drop(columns=drop_cols, inplace=True)

In [172]:
d_man= {}
for key, company in enumerate(df['oem'].unique()):
    d_man[key] = company
    

In [174]:
man_list = list(df['oem'].unique())

man_list

['Acer',
 'alcatel',
 'Allview',
 'Amazon',
 'Amoi',
 'Apple',
 'Archos',
 'Asus',
 'AT&T',
 'Benefon',
 'BenQ',
 'BenQ-Siemens',
 'Bird',
 'BlackBerry',
 'Blackview',
 'BLU',
 'Bosch',
 'BQ',
 'Casio',
 'Cat',
 'Celkon',
 'Coolpad',
 'Dell',
 'Emporia',
 'Energizer',
 'Ericsson',
 'Eten',
 'Fujitsu Siemens',
 'Garmin-Asus',
 'Gigabyte',
 'Gionee',
 'Google',
 'Haier',
 'Honor',
 'HP',
 'HTC',
 'Huawei',
 'i-mate',
 'i-mobile',
 'Icemobile',
 'Infinix',
 'Innostream',
 'iNQ',
 'Intex',
 'Jolla',
 'Karbonn',
 'Kyocera',
 'Lava',
 'LeEco',
 'Lenovo',
 'LG',
 'Maxon',
 'Maxwest',
 'Meizu',
 'Micromax',
 'Microsoft',
 'Mitac',
 'Mitsubishi',
 'Modu',
 'Motorola',
 'MWg',
 'NEC',
 'Neonode',
 'NIU',
 'Nokia',
 'Nvidia',
 'O2',
 'OnePlus',
 'Oppo',
 'Orange',
 'Palm',
 'Panasonic',
 'Pantech',
 'Parla',
 'Philips',
 'Plum',
 'Posh',
 'Prestigio',
 'QMobile',
 'Qtek',
 'Razer',
 'Realme',
 'Sagem',
 'Samsung',
 'Sendo',
 'Sewon',
 'Sharp',
 'Siemens',
 'Sonim',
 'Sony',
 'Sony Ericsson',
 'Sp

In [160]:
d_manufacturer
man_df = pd.DataFrame(d_manufacturer.)
man_df

,0,1
0,0,Acer
1,1,alcatel
2,2,Allview
3,3,Amazon
4,4,Amoi
5,5,Apple
6,6,Archos
7,7,Asus
8,8,AT&T
9,9,Benefon


In [115]:
df_acer = df.loc[df['oem'] == 'Acer', 'main_camera_video']
df_acer

0     720
1     Yes
2     Yes
3     Yes
4     Yes
5     Yes
6     Yes
7     Yes
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13    Yes
14    Yes
15    Yes
16    Yes
17    Yes
18    Yes
19    Yes
20    Yes
21    Yes
22    Yes
23    Yes
24    Yes
25    Yes
26    Yes
27    Yes
28    Yes
29    Yes
     ... 
64    720
65    720
66    720
67    720
68    Yes
69    Yes
70    Yes
71    Yes
72    Yes
73    Yes
74    Yes
75    Yes
76    Yes
77    Yes
78    Yes
79    Yes
80    Yes
81    Yes
82    Yes
83    Yes
84    Yes
85    Yes
86    Yes
87    Yes
88    Yes
89    NaN
90    Yes
91    480
92    Yes
93    Yes
Name: main_camera_video, Length: 94, dtype: object

In [116]:
analysis_drop_list = ['is_watch', 'is_tablet']

In [117]:
def prep_analysis(data_frame, col_list):
    
    data_frame.drop(columns=col_list, inplace=True)
    
    return data_frame

In [118]:
# df.drop(columns = analysis_drop_list, inplace = True)

In [168]:
df_phones = df.loc[~df.is_tablet, :]
df_phones = df_phones.loc[~df_phones.is_watch, :]

In [169]:
df_phones.head(5)

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple,is_tablet,is_watch,sensor_accelerometer,sensor_gyro,sensor_heart_rate,sensor_fingerprint,sensor_compass,sensor_proximity,sensor_barometer,sensor_spo2,sensor_iris_scanner,sensor_gesture,sensor_tempurature,sensor_altimeter,sensor_infrared_face_recognition
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","13 MP, AF",Yes,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","13 MP, AF",Yes,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Fingerprint (front-mounted), accelerometer, pr...",About 250 EUR,HSDPA,LTE,NaN,Mali-T720MP3,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False
3,Acer,Liquid Z6,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,126 g (4.44 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"IPS LCD capacitive touchscreen, 16M colors","5.0 inches, 68.9 cm (~65.3% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6737 (28 nm),Quad-core 1.25 GHz Cortex-A53,"microSD, up to 256 GB","8 MP, AF",Yes,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 120 EUR,HSDPA,LTE,NaN,Mali-T720MP2,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False
5,Acer,Liquid X2,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,Yes,Yes,2015-04,166 g (5.86 oz),Triple SIM (Micro-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~69.0% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~267 ppi density)",Android 5.1 (Lollipop),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 32 GB (dedicated slot)","13 MP, AF, f/1.8",Yes,"13 MP, f/1.8, AF",NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP, LE","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity, compass",About 230 EUR,"HSDPA 900 / 1900 / 2100 - Europe,

In [170]:
# df = df.loc[df.network_technology != np.nan, :]
df_phones = df_phones[df_phones['network_technology'].notna()]

In [171]:
df_phones.shape

(8563, 61)

In [123]:
# df.groupby([df.datetime.dt.date, 'rating']).signal.nunique().eq(1)

In [124]:
# df.groupby([df['oem'], 'main_camera_video']).count()

In [125]:
df_samsung = df[df['oem'] == 'Samsung'].sort_values(by='launch_announced')

In [126]:
df_samsung.head(5)


,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple,is_tablet,is_watch,sensor_accelerometer,sensor_gyro,sensor_heart_rate,sensor_fingerprint,sensor_compass,sensor_proximity,sensor_barometer,sensor_spo2,sensor_iris_scanner,sensor_gesture,sensor_tempurature,sensor_altimeter,sensor_infrared_face_recognition
7277,Samsung,SGH-500,GSM,GSM 900,NaN,NaN,1998-07,115g with slim battery (4.06 oz),Mini-SIM,NaN,Monochrome graphic,NaN,"78 x 42 pixels, 4 lines, 17:9 ratio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V1,NaN,NaN,NaN,NaN,NaN,NaN,6 voice languages,Voice dial Speed dial Menu shortcuts World ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7271,Samsung,SGH-2400,GSM,GSM 900 / 1800,NaN,NaN,1999-07,90 g with slim battery (3.17 oz),Mini-SIM,NaN,Monochrome graphic,NaN,"115 x 60 pixels, 4 lines, 19:10 ratio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voice dial Voice memo Speed dial Built-in t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7272,Samsung,SGH-2200,GSM,GSM 900 / 1800,NaN,NaN,1999-07,111 g with slim battery (3.92 oz),Mini-SIM,NaN,Monochrome graphic,NaN,"115 x 60 pixels, 4 lines, 19:10 ratio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voice dial Voice memo Speed dial Manner Fun...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7276,Samsung,SGH-600,GSM,GSM 900,NaN,NaN,1999-07,95 g with slim battery (3.35 oz),Mini-SIM,NaN,Monochrome graphic,NaN,"96 x 32 pixels, 4 lines",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voice dial Voice memo Speed dial Menu short...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7273,Samsung,SGH-2100,GSM,GSM 900 / 1800,NaN,NaN,1999-07,111 g with slim battery (3.92 oz),Mini-SIM,NaN,Monochrome graphic,NaN,"115 x 60 pixels, 4 lines, 19:10 ratio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voice dial Voice memo Speed dial Menu short...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [127]:
# df_samsung['main_camera_video'].first(1)

In [128]:
# def return_first(value, date):
#     if type(value)!=str or type(value)!=int:
#         if math.isnan(value):
#             pass
#     else:
#         return date

In [129]:
# test = df_samsung.apply(lambda x: 
#                         return_first(x['main_camera_video'], 
#                                      x['launch_announced']), axis=0)

In [131]:
# test = df_samsung.apply(lambda df_samsung: 
#                         return_first(df_samsung['main_camera_video'], 
#                                      df_samsung['launch_announced']), axis=1)

In [143]:
# df_samsung['main_camera_video']

In [144]:
df_samsung['main_camera_video'].first_valid_index()


7236

In [147]:
df_samsung.loc[df_samsung['main_camera_video'].first_valid_index(), 'launch_announced']

'2003-11'

In [177]:
feature_cols = ['oem', 'feat_released_month', 'months_after_release', 
                'feat_removed_month', 'months_before_release']

In [ ]:
d_feature[